# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 47 new papers today
          19 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/18 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.05236


extracting tarball to tmp_2301.05236...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.05244


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.05236/suppinfo.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2301.05616...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.05699


extracting tarball to tmp_2301.05699...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.05699/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05236-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05236) | **End-to-end study of the home and genealogy of the first binary neutron  star merger**  |
|| <mark>Heloise F. Stevance</mark>, et al. |
|*Appeared on*| *2023-01-16*|
|*Comments*| *Main text (3 Figures, 10 pages) and Supplementary Information (23 pages, 14 Figures). Published in Nature Astronomy*|
|**Abstract**| Binary neutron star mergers are one of the ultimate events of massive binary star evolution, and our understanding of their parent system is still in its infancy. Upcoming gravitational wave detections, coupled with multi-wavelength follow-up observations, will allow us to study an increasing number of these events by characterising their neighbouring stellar populations and searching for their progenitors. Stellar evolution simulations are essential to this work but they are also based on numerous assumptions. Additionally, the models used to study the host galaxies differ from those used to characterise the progenitors and are typically based on single star populations. Here we introduce a framework to perform an end-to-end analysis and deploy it to the first binary neutron star merger - GW170817. With the Binary Population And Spectral Synthesis (BPASS) codes we are able to retrieve the physical properties of the host galaxy NGC 4993 as well as infer progenitor candidates. In our simulations there is a >98% chance that GW170817 originated from a stellar population with Z=0.010 born between 5 and 12.5 Gyrs ago. By carefully weighing the stellar genealogies we find that GW170817 most likely came from a binary system born with a 13-24 Msol primary and 10-12 Msol secondary which underwent two or three common envelope events over their lifetime. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05244-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05244) | **Strong gravitational lensing's `external shear' is not shear**  |
|| Amy Etherington, et al. -- incl., <mark>Qiuhan He</mark>, <mark>Ran Li</mark>, <mark>Marceau Limousin</mark>, <mark>Guillaume Mahler</mark> |
|*Appeared on*| *2023-01-16*|
|*Comments*| *13 pages, 11 figures, submitted to MNRAS*|
|**Abstract**| The distribution of mass in galaxy-scale strong gravitational lenses is often modelled as an elliptical power law plus `external shear', which notionally accounts for neighbouring galaxies and cosmic shear. We show that it does not. Except in a handful of rare systems, the best-fit values of external shear do not correlate with independent measurements of shear: from weak lensing in 45 Hubble Space Telescope images, or in 50 mock images of lenses with complex distributions of mass. Instead, the best-fit shear is aligned with the major or minor axis of 88% of lens galaxies; and the amplitude of the external shear increases if that galaxy is disky. We conclude that `external shear' attached to a power law model is not physically meaningful, but a fudge to compensate for lack of model complexity. Since it biases other model parameters that are interpreted as physically meaningful in several science analyses (e.g. measuring galaxy evolution, dark matter physics or cosmological parameters), we recommend that future studies of galaxy-scale strong lensing should employ more flexible mass models. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05251-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05251) | **CO Excitation in High-z Main Sequence Analogues: Resolved  CO(4-3)/CO(3-2) Line Ratios in DYNAMO Galaxies**  |
|| Laura Lenkić, et al. -- incl., <mark>Rodrigo Herrera-Camus</mark> |
|*Appeared on*| *2023-01-16*|
|*Comments*| *22 pages, 9 figures, 4 tables, ApJ accepted*|
|**Abstract**| The spectral line energy distribution of carbon monoxide contains information about the physical conditions of the star forming molecular hydrogen gas; however, the relation to local radiation field properties is poorly constrained. Using ~ 1-2 kpc scale ALMA observations of CO(3-2) and CO(4-3), we characterize the CO(4-3)/CO(3-2) line ratios of local analogues of main sequence galaxies at z ~ 1-2, drawn from the DYNAMO sample. We measure CO(4-3)/CO(3-2) across the disk of each galaxy and find a median line ratio of $R_{43} = 0.54^{+0.16}_{-0.15}$ for the sample. This is higher than literature estimates of local star-forming galaxies and is consistent with multiple lines of evidence that indicate DYNAMO galaxies, despite residing in the local Universe, resemble main-sequence galaxies at z ~ 1-2. Comparing to existing lower resolution CO(1-0) observations, we find $R_{41}$ and $R_{31}$ values in the range $\sim 0.2-0.3$ and $\sim 0.4-0.8$ respectively. We combine our kpc-scale resolved line ratio measurements with HST observations of H$\alpha$ to investigate the relation to star formation rate surface density and compare this relation to expectations from models. We find increasing CO(4-3)/CO(3-2) with increasing star formation rate surface density; however, models over-predict the line ratios across the range of star formation rate surface densities we probe, particularly at the lower range. Finally, SOFIA observations with HAWC+ and FIFI-LS reveal low dust temperatures and no deficit of [CII] emission with respect to the total infrared luminosity. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05509-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05509) | **New upper limits on low-frequency radio emission from isolated neutron  stars with LOFAR**  |
|| Inés Pastor-Marazuela, Samayra M. Straal, <mark>Joeri van Leeuwen</mark>, Vlad I. Kondratiev |
|*Appeared on*| *2023-01-16*|
|*Comments*| *7 pages, 2 figures, 1 table. Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| Neutron stars that show X-ray and $\gamma$-ray pulsed emission must, somewhere in the magnetosphere, generate electron-positron pairs. Such pairs are also required for radio emission, but then why do a number of these sources appear radio quiet? Here, we carried out a deep radio search towards four such neutron stars that are isolated X-ray/$\gamma$-ray pulsars but for which no radio pulsations have been detected yet. These sources are 1RXS J141256.0+792204 (Calvera), PSR J1958+2846, PSR J1932+1916 and SGR J1907+0919. Searching at lower radio frequencies, where the radio beam is thought to be wider, increases the chances of detecting these sources, compared to the earlier higher-frequency searches. We thus carried a search for periodic and single-pulse radio emission with the LOFAR radio telescope at 150 MHz. We used the known periods, and searched a wide range of dispersion measures, as the distances are not well constrained. We did not detect pulsed emission from any of the four sources. However, we put very constraining upper limits on the radio flux density at 150 MHz, of $\lesssim$1.4 mJy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05520-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05520) | **Toward accurate measurement of property-dependent galaxy clustering: II.  Tests of the smoothed density-corrected $V_{\rm max}$ method**  |
|| Lei Yang, <mark>Zhigang Li</mark> |
|*Appeared on*| *2023-01-16*|
|*Comments*| *19 pages, 12 figures. Accepted for publication in ApJ*|
|**Abstract**| We present a smoothed density-corrected $V_{\rm max}$ technique for building a random catalog for property-dependent galaxy clustering estimation. This approach is essentially based on the density-corrected $V_{\rm max}$ method of Cole(2011), with three improvements to the original method. To validate the improved method, we generate two sets of flux-limited samples from two independent mock catalogs with different $k+e$ corrections. By comparing the two-point correlation functions, our results demonstrate that the random catalog created by the smoothed density-corrected $V_{\rm max}$ approach provides a more accurate and precise measurement for both sets of mock samples than the commonly used $V_{\rm max}$ method and redshift shuffled method. For flux-limited samples and color-dependent subsamples, the accuracy of the projected correlation function is well constrained within $1\%$ on the scale $0.07 h^{-1}\rm Mpc$ to $30 h^{-1}\rm Mpc$. The accuracy of the redshift-space correlation function is less than $2\%$ as well. Currently, it is the only approach that holds promise for achieving the high-accuracy goal of clustering measures for next-generation surveys. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05616-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05616) | **First Perihelion of EUI on the Solar Orbiter mission**  |
|| D. Berghmans, et al. -- incl., <mark>P. J. Smith</mark>, <mark>K. Heerlein</mark> |
|*Appeared on*| *2023-01-16*|
|*Comments*| **|
|**Abstract**| Context. The Extreme Ultraviolet Imager (EUI), onboard Solar Orbiter consists of three telescopes: the two High Resolution Imagers in EUV (HRIEUV) and in Lyman-{\alpha} (HRILya), and the Full Sun Imager (FSI). Solar Orbiter/EUI started its Nominal Mission Phase on 2021 November 27. Aims. EUI images from the largest scales in the extended corona off limb, down to the smallest features at the base of the corona and chromosphere. EUI is therefore a key instrument for the connection science that is at the heart of the Solar Orbiter mission science goals. Methods. The highest resolution on the Sun is achieved when Solar Orbiter passes through the perihelion part of its orbit. On 2022 March 26, Solar Orbiter reached for the first time a distance to the Sun close to 0.3 au. No other coronal EUV imager has been this close to the Sun. Results. We review the EUI data sets obtained during the period 2022 March-April, when Solar Orbiter quickly moved from alignment with the Earth (2022 March 6), to perihelion (2022 March 26), to quadrature with the Earth (2022 March 29). We highlight the first observational results in these unique data sets and we report on the in-flight instrument performance. Conclusions. EUI has obtained the highest resolution images ever of the solar corona in the quiet Sun and polar coronal holes. Several active regions were imaged at unprecedented cadences and sequence durations. We identify in this paper a broad range of features that require deeper studies. Both FSI and HRIEUV operate at design specifications but HRILya suffered from performance issues near perihelion. We conclude emphasising the EUI open data policy and encouraging further detailed analysis of the events highlighted in this paper. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05699-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05699) | **Multi-Wavelength and Multi-Messenger Studies with the next-generation  Event Horizon Telescope**  |
|| <mark>Rocco Lico</mark>, et al. -- incl., <mark>Ioannis Liodakis</mark> |
|*Appeared on*| *2023-01-16*|
|*Comments*| *10 pages, 3 figures*|
|**Abstract**| The next-generation Event Horizon Telescope (ngEHT) will provide us with the best opportunity to investigate supermassive black holes (SMBHs) at the highest possible resolution and sensitivity. With respect to the existing Event Horizon Telescope (EHT) array, the ngEHT will provide increased sensitivity and uv-coverage with the addition of new stations, wider frequency coverage (from 86 GHz to 345 GHz and higher), finer resolution (<15 micro-arcseconds), and better monitoring capabilities. This will offer a unique opportunity to deeply investigate the physics around SMBHs, such as the disk-jet connection, the mechanisms responsible for high-energy photon and neutrino events, the role of magnetic fields in shaping relativistic jets, as well as the nature of binary SMBH systems. In this white paper we describe some ngEHT science cases in the context of multi-wavelength studies and synergies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05532-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05532) | **The Fornax3D project: Environmental effects on the assembly of  dynamically cold disks in Fornax cluster galaxies**  |
|| Y. Ding, et al. -- incl., <mark>F. Pinna</mark> |
|*Appeared on*| *2023-01-16*|
|*Comments*| *31 pages, 32 figures, Accepted to A&A*|
|**Abstract**| We apply a population-orbit superposition method to 16 galaxies in the Fornax cluster observed with MUSE/VLT in the context of the Fornax3D project. By fitting the luminosity distribution, stellar kinematics, and age and metallicity maps simultaneously, we obtained the internal stellar orbit distribution, as well as the age and metallicity distribution of stars on different orbits for each galaxy. Based on the model, we decompose each galaxy into a dynamically cold disk (orbital circularity $\lambda_z\ge0.8$) and a dynamically hot non-disk component (orbital circularity $\lambda_z<0.8$), and obtain the surface-brightness, age, and metallicity radial profiles of each component. The galaxy infall time into the cluster is strongly correlated with galaxy cold-disk age with older cold disks in ancient infallers. We quantify the infall time $t_{\rm infall}$ of each galaxy with its cold-disk age using a correlation calibrated with TNG50 cosmological simulations. For galaxies in the Fornax cluster, we found that the luminosity fraction of cold disk in galaxies with $t_{\rm infall}>8$ Gyr are a factor of $\sim 4$ lower than in more recent infallers while controlling for total stellar mass. Nine of the 16 galaxies have spatially extended cold disks, and most of them show positive or zero age gradients; stars in the inner disk are $\sim 2-5$ Gyr younger than that in the outer disk, in contrast to the expectation of inside-out growth. Our results indicate that the assembly of cold disks in galaxies is strongly affected by their infall into clusters, by either removal of gas in outer regions or even tidally stripping or heating part of the pre-existing disks. Star formation in outer disks can stop quickly after the galaxy falls into the cluster, while star formation in the inner disks can last for a few Gyrs more, building the positive age gradient measured in cold disks. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure .png</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

167  publications in the last 7 days.
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers